# PurpleAir Stations QAQC

## Import Packages

In [3]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

In [18]:
# Load our Functions

script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())
# exec(open(os.path.join(script_path, 'Update_Alerts.py')).read())

## Definitions

In [19]:
# Getting .env information (PurpleAir API Read Key)
from dotenv import load_dotenv

load_dotenv()

api = os.getenv('PURPLEAIR_API_TOKEN')

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds)) 

## Load Previous Information

### Sensor Information

Want sensor_index, channel_flags, last_seen

In [5]:
# Connect

conn = psycopg2.connect(**pg_connection_dict) 
# Create cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT sensor_index, channel_flags, last_seen 
FROM "PurpleAir Stations"
''')

cur.execute(cmd) # Execute
conn.commit() # Committ command

# Unpack response into pandas series

sensors_df = pd.DataFrame(cur.fetchall(), columns = ['sensor_index', 'channel_flags', 'last_seen'])

# Close cursor
cur.close()
# Close connection
conn.close()


In [20]:
sensors_df.head()

,sensor_index,channel_flags,last_seen
0,142718,0,2023-11-02 19:07:56
1,142720,0,2023-11-02 19:08:13
2,142726,0,2023-11-02 19:07:38
3,142724,0,2023-11-02 19:08:16
4,142730,0,2023-09-25 09:18:30


### Extent of study

Getting lat/long for PurpleAir API Parameters

In [10]:
# Connect

conn = psycopg2.connect(**pg_connection_dict) 
# Create cursor
cur = conn.cursor()

cmd = sql.SQL('''
WITH buffer as
	(
	SELECT ST_BUFFER(ST_Transform(ST_SetSRID(geometry, 4326),
								  26915),
					 100) geom -- buff the geometry by 100 meters
	FROM "Minneapolis Boundary"
	), bbox as
	(
	SELECT ST_EXTENT(ST_Transform(geom, 4326)) b
	FROM buffer
	)
SELECT b::text
FROM bbox;
''')

cur.execute(cmd) # Execute
conn.commit() # Committ command

In [11]:
# Gives a string

response = cur.fetchall()[0][0]

# Close cursor
cur.close()
# Close connection
conn.close()

In [14]:
response

'BOX(-93.33037537752216 44.88968834134478,-93.19306250738248 45.05214646628739)'

In [15]:
# Unpack the response

num_string = response[4:-1]

num_string 

'-93.33037537752216 44.88968834134478,-93.19306250738248 45.05214646628739'

In [16]:
# That's in xmin, ymin, xmax, ymax

xmin = num_string.split(' ')[0]
ymin = num_string.split(' ')[1].split(',')[0]
xmax = num_string.split(' ')[1].split(',')[1]
ymax = num_string.split(' ')[2]

# Convert into PurpleAir API notation

nwlng, selat, selng, nwlat = xmin, ymin, xmax, ymax

## Importing New PurpleAir Station Data

In [21]:
#Set bounding strings for API parameters
bounds_strings = [f'nwlng={nwlng}',
                  f'nwlat={nwlat}',
                  f'selng={selng}',
                  f'selat={selat}']

bounds_string = '&'.join(bounds_strings)

print(bounds_string)

nwlng=-93.33037537752216&nwlat=45.05214646628739&selng=-93.19306250738248&selat=44.88968834134478


In [31]:
#Setting parameters for API

# Fields
fields = ['sensor_index', 'channel_flags', 'last_seen', 'name']

fields_string = 'fields=' + '%2C'.join(fields)

# Finalizing query for API function
query_string = '&'.join([fields_string, bounds_string])

In [32]:
#calling the API
response = getSensorsData(query_string, api)

In [33]:
# Unpack response

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

df = pd.DataFrame(data, columns = col_names) # Convert to dataframe

# Correct Last Seen

df['last_seen'] = pd.to_datetime(sensors_df['last_seen'],
                                         utc = True,
                                         unit='s').dt.tz_convert('America/Chicago')

# Filter for City of Minneapolis

is_city = df.name.apply(lambda x: 'CITY OF MINNEAPOLIS' in x.upper())

df_city =  df[is_city]

In [34]:
# See results

df_city

,sensor_index,last_seen,name,channel_flags
13,142718,2023-06-22 05:19:40-05:00,City of Minneapolis Community Air Monitoring P...,0
14,142720,2023-10-19 06:03:33-05:00,City of Minneapolis Community Air Monitoring P...,0
15,142726,2023-11-02 14:09:21-05:00,City of Minneapolis Community Air Monitoring P...,0
16,142724,2023-03-28 09:34:07-05:00,City of Minneapolis Community Air Monitoring P...,0
17,142734,2022-11-10 05:26:39-06:00,City of Minneapolis Community Air Monitoring P...,0
18,142732,2023-11-02 14:08:57-05:00,City of Minneapolis Community Air Monitoring P...,1
19,142738,2023-11-02 14:08:33-05:00,City of Minneapolis Community Air Monitoring P...,0
20,142736,2023-06-24 18:07:54-05:00,City of Minneapolis Community Air Monitoring P...,0
21,142742,2023-10-31 15:05:32-05:00,City of Minneapolis Community Air Monitoring ...,0
22,142744,2023-11-02 14:09:13-05:00,City of Minneapolis Community Air Monitoring P...,0


In [14]:
# From what we just queried from PurpleAir 
# Only want the City's monitors

is_city = df.name.apply(lambda x: 'CITY OF MINNEAPOLIS' in x.upper())

ids_from_PurpleAir = df[is_city].sensor_index.astype(int)

In [15]:
len(ids_from_PurpleAir)

48

In [16]:
# Now let's get the union of these two lists of ids

ids_from_either = np.union1d(ids_from_previous_run, ids_from_PurpleAir)

In [17]:
len(ids_from_either)

66

### Final PurpleAir Query

In [18]:
# One final query for this list of ids

# Get start Times

sensor_string = 'show_only=' + '%2C'.join(ids_from_either.astype(str))

query_string = '&'.join([fields_string+'%2Cpm2.5_10minute', sensor_string])

In [19]:
response = getSensorsData(query_string, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=firmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cname%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude%2Cpm2.5_10minute&show_only=142718%2C142720%2C142724%2C142726%2C142728%2C142730%2C142732%2C142734%2C142736%2C142738%2C142744%2C142748%2C142750%2C142752%2C142756%2C142772%2C142774%2C142926%2C143214%2C143216%2C143222%2C143224%2C143226%2C143238%2C143240%2C143242%2C143246%2C143248%2C143634%2C143636%2C143648%2C143656%2C143660%2C143666%2C143668%2C143916%2C143942%2C143944%2C145202%2C145204%2C145242%2C145250%2C145454%2C145470%2C145498%2C145502%2C145504%2C145506%2C145604%2C145610%2C145614%2C145616%2C156605%2C157747%2C157757%2C157785%2C157837%2C157845%2C157861%2C157871%2C157877%2C157935%2C166459%2C168327%2C177765%2C194635


## Cleaning PurpleAir Station Data

In [20]:
#find outside sensors
df_outside = sensors_df[sensors_df.location_type == 0]
len(df_outside)

66

In [21]:
#drop the location_type now that we have filtered for outdoor sensors only
df_stations = df_outside.drop('location_type', axis=1)

df_stations['city_id'] = our_sensor_info['name'].apply(lambda x: x.split(' ')[-1])

In [22]:
#Converting UNIX date/time to pd 

# IN UTC time (5 hours ahead of here)

df_stations['last_modified'] = pd.to_datetime(sensors_df['last_modified'],
                                         utc = True,
                                         unit='s').dt.tz_convert('America/Chicago')
df_stations['date_created'] = pd.to_datetime(sensors_df['date_created'],
                                         utc = True,
                                         unit='s').dt.tz_convert('America/Chicago')
df_stations['last_seen'] = pd.to_datetime(sensors_df['last_seen'],
                                         utc = True,
                                         unit='s').dt.tz_convert('America/Chicago')

In [23]:
df_stations.head(3)

,sensor_index,last_modified,date_created,last_seen,name,firmware_version,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags,pm2.5_10minute,city_id
0,142718,2023-08-02 13:50:17,2022-01-12 13:57:49,2023-11-02 19:07:56,City of Minneapolis Community Air Monitoring P...,7.02,8753,5,44.99631,-93.29565,857,3,0,48.0,50
1,142720,2023-08-02 13:06:36,2022-01-12 13:57:55,2023-11-02 19:08:13,City of Minneapolis Community Air Monitoring P...,7.02,28985,5,44.955555,-93.254974,850,3,0,30.9,29
2,142726,2023-02-02 12:31:06,2022-01-12 13:58:17,2023-11-02 19:07:38,City of Minneapolis Community Air Monitoring P...,7.02,4937,5,45.01507,-93.28903,889,3,0,34.8,11


In [24]:
# Rename a column and the dateframe for ease

df = df_stations.rename(columns = {'pm2.5_10minute':'pm25'})

In [25]:
# Save as CSV

df_stations.to_csv(os.path.join(datapath, 'PurpleAir_Stations.csv'))

# Save as geojson

gdf_stations = gpd.GeoDataFrame(df_stations, 
                                geometry = gpd.points_from_xy(
                                    df_stations.longitude,
                                    df_stations.latitude,
                                    crs = 'EPSG:4326')
                               ).to_crs('EPSG:26915')

cols_for_db = ['sensor_index'] + fields[:-3] + ['geometry']

sorted_gdf = gdf_stations[cols_for_db]

sorted_gdf.to_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

In [26]:
# Let's explore

pd.set_option('display.max_colwidth', None)

In [41]:
# Our Sensor Flags

# Key
# Channel State -  0 = No PM, 3 = Both On
# Channel Flags - 0 = Normal, 1 = A Downgraded, 2 - B Downgraded, 3 - Both Downgraded

flagged = (df.channel_flags != 0
          ) | (df.channel_state == 0
              ) |(df.last_seen < dt.datetime.now() - dt.timedelta(minutes=60)
                 )

flagged_sensors = df[flagged][['sensor_index', 
                               'channel_flags', 
                               'channel_state',
                                'last_seen',
                               'name',
                               'city_id','pm25']]

not_flagged_sensors = df[~flagged][['sensor_index', 
                               'channel_flags',
                               'channel_state',
                                'last_seen',
                               'name',
                                    'city_id', 'pm25']]

print('Our Flags: ', len(flagged_sensors))
# Save the flagged sensors
# flagged_sensors.sort_values('last_seen').reset_index(drop=True).to_csv(os.path.join(datapath,'flagged_sensors.csv'))

# flagged_sensors.sort_values('last_seen')

Our Flags:  29


In [42]:
# Load the list the City provided

city_sensor_info = pd.read_excel(os.path.join(datapath, 'Updated PurpleAir IDs and locations.xlsx')) # Load as DataFrame

city_sensor_info['city_id'] = city_sensor_info['Location Name'].apply(lambda x: x.split(' ')[-1])

city_sensor_info = city_sensor_info.fillna('No Notes')

In [29]:
# For checking with the City's list

city_names_number = set(city_sensor_info.city_id)
our_names_number = set(our_sensor_info.city_id)

print("city_ids we have the city doesn't have", our_names_number - city_names_number)
print('\n')
print("city_ids City has that we don't have", city_names_number - our_names_number)
print('\n')
print("Length of City's list", len(city_names_number))

city_ids we have the city doesn't have {'51', '64', '31', 'Minneapolis', '28'}


city_ids City has that we don't set()


Length of City's list 59


In [30]:
# Compare further with the city

# Merge

merged_df = pd.merge(df, city_sensor_info,
                    on='city_id',
                   how='outer',
                    suffixes = ['_ours', '_city'])

In [39]:
# Compare flags

flags = (merged_df.channel_flags != 0
          ) | (merged_df.channel_state == 1
              ) |(merged_df.last_seen < dt.datetime.now() - dt.timedelta(minutes=60)
                 )

flags_df = merged_df[flags][['city_id',
                               'sensor_index', 
                               'channel_flags', 
                               'channel_state',
                               'Notes',
                                'last_seen',
                               'name',
                               'pm25']]

flags_df[['city_id', 'last_seen', 'Notes', 'channel_flags', 'channel_state']].sort_values('last_seen')#.to_csv('flagged_sensors_1910.csv')

,city_id,last_seen,Notes,channel_flags,channel_state
17,22,2022-11-10 11:26:39,disconnected (offline),0,3
48,4,2023-01-09 17:09:39,disconnected (offline),0,3
16,31,2023-03-28 14:34:07,NaN,0,3
65,28,2023-03-28 14:37:14,NaN,3,0
24,35,2023-04-26 15:33:55,disconnected (offline),0,3
13,40,2023-06-22 10:19:40,disconnected (offline),0,3
21,Minneapolis,2023-06-24 23:07:54,NaN,0,3
38,37,2023-06-27 07:01:58,disconnected (offline),0,3
39,60,2023-06-29 14:20:54,disconnected (offline),0,3
50,24,2023-07-03 06:22:30,disconnected (offline),0,3


## Insert Data into SQL Table - Not Quite Done

This doesn't check to see if the sensors are already in the database

In [33]:
cols_for_db

['sensor_index',
 'firmware_version',
 'date_created',
 'last_modified',
 'last_seen',
 'name',
 'uptime',
 'position_rating',
 'channel_state',
 'channel_flags',
 'altitude',
 'geometry']

In [34]:
# Sort the df and correct values

sorted_df = sorted_gdf.drop('geometry', axis=1).copy()

sorted_df['wkt'] = sorted_gdf.geometry.apply(lambda x: x.wkt)

sorted_df['date_created'] = sorted_gdf.date_created.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))
sorted_df['last_modified'] = sorted_gdf.last_modified.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))
sorted_df['last_seen'] = sorted_gdf.last_seen.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))

# Replace non-numeric nans

# sorted_df['firmware_version'] = sorted_df['firmware_version'].fillna(value = 'a', inplace = True)
# sorted_df['firmware_version'] = sorted_df['firmware_version'].fillna(value = 'NaN', inplace = True)

In [35]:
sorted_df.head()

,sensor_index,firmware_version,date_created,last_modified,last_seen,name,uptime,position_rating,channel_state,channel_flags,altitude,wkt
0,142718,7.02,2022-01-12 13:57:49,2023-08-02 13:50:17,2023-11-02 19:07:56,City of Minneapolis Community Air Monitoring Project 50,8753,5,3,0,857,POINT (476696.84844637447 4982582.8198003275)
1,142720,7.02,2022-01-12 13:57:55,2023-08-02 13:06:36,2023-11-02 19:08:13,City of Minneapolis Community Air Monitoring Project 29,28985,5,3,0,850,POINT (479888.6697801179 4978044.582460884)
2,142726,7.02,2022-01-12 13:58:17,2023-02-02 12:31:06,2023-11-02 19:07:38,City of Minneapolis Community Air Monitoring Project 11,4937,5,3,0,889,POINT (477226.07726585865 4984664.938670933)
3,142724,7.02,2022-01-12 13:58:09,2023-08-02 11:12:05,2023-11-02 19:08:16,City of Minneapolis Community Air Monitoring Project 21,3448,5,3,0,859,POINT (480783.00497330126 4976062.035271977)
4,142730,7.02,2022-01-12 13:58:36,2023-08-02 10:41:14,2023-09-25 09:18:30,City of Minneapolis Community Air Monitoring Project 13,816,5,3,0,858,POINT (476695.09256082535 4982102.367948439)


In [37]:
# Connect to PostGIS Database

conn = psycopg2.connect(**pg_connection_dict)
cur = conn.cursor()


# iterate over the dataframe and insert each row into the database using a SQL INSERT statement

for index, row in sorted_df.copy().iterrows():

    q1 = sql.SQL('INSERT INTO "PurpleAir Stations" ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_Transform(ST_SetSRID(ST_GeomFromText(%s), 26915),4326)::geometry'))
    # print(q1.as_string(conn))
    # print(row)
    # break
    
    cur.execute(q1.as_string(conn),
        (list(row.values))
        )
    # Commit command

    conn.commit()

# Close the cursor and connection
cur.close()
conn.close()